In [6]:
import pygame as pg
import os 
import time 
import utils as ut
BLUE = (0,80,239)
BLACK = (0,0,0)
GREEN = (0,138,0)
RED = (229,20,0)
WHITE = (255,255,255)

pg.init()

clock = pg.time.Clock()
screen = pg.display.set_mode((1600,900))
pg.display.set_caption("Chapelié")


[flag_be, flag_fr, flag_rma, flag_x] = ut.prepare_flags()

flag_school_dim = (flag_rma[0].get_width(),flag_rma[0].get_height())
flag_country_dim = (flag_be[0].get_width(),flag_be[0].get_height())


boat = pg.transform.scale(pg.image.load(os.path.join(".", "figures","warship.png")),(375,375))
background = pg.transform.scale(pg.image.load(os.path.join(".", "figures","canal.png")),(4000,2000)).convert()
background.set_alpha(128)

[font_30, font_50, font_70] = ut.import_fonts() 


count = 0
start = time.time()
for i in range(200):
    duration = time.time()-start
    if count%200 < 24 and count > 100:
        if round(duration,1)%0.4 < 0.2:
            color = GREEN
        else:
            color = WHITE
    else:
        color = WHITE
    
    screen.blit(background,((-count*4)%-background.get_width(),-550),(0,0,background.get_width(),background.get_height()//2))
    screen.blit(background,((-count*4)%-background.get_width()+background.get_width(),-550),(0,0,background.get_width(),background.get_height()//2))
    screen.blit(background,((-count*4)%-background.get_width(),screen.get_height()//2),(0,background.get_height()//2,background.get_width(),background.get_height()//2))
    screen.blit(background,((-count*4)%-background.get_width()+background.get_width(),screen.get_height()//2),(0,background.get_height()//2,background.get_width(),background.get_height()//2))
    
    for i in range(count//200-count//1000,count//200-count//1000+3):
        distance_marker = pg.transform.rotate(font_30.render(str(i*200/1e3)+"km", True, color),-90)
        distance_marker_rect = distance_marker.get_rect()
        distance_marker_rect.midleft = (i*200*5-count*4,325)
        screen.blit(distance_marker,distance_marker_rect)
    
    for i in range(count//200-count//1000,count//200-count//1000+3):
        distance_marker = pg.transform.rotate(font_30.render(str(i*200/1e3)+"km", True, color),-90)
        distance_marker_rect = distance_marker.get_rect()
        distance_marker_rect.midleft = (i*200*5-count*4,screen.get_height()-325)
        screen.blit(distance_marker,distance_marker_rect)

    screen.blit(background,((-count*4)%-background.get_width(),-550),(0,0,background.get_width(),background.get_height()//2))
    screen.blit(background,((-count*4)%-background.get_width()+background.get_width(),-550),(0,0,background.get_width(),background.get_height()//2))
    
    screen.blit(background,((-count*4)%-background.get_width(),screen.get_height()//2),(0,background.get_height()//2,background.get_width(),background.get_height()//2))
    screen.blit(background,((-count*4)%-background.get_width()+background.get_width(),screen.get_height()//2),(0,background.get_height()//2,background.get_width(),background.get_height()//2))
    
    screen.blit(flag_rma[(count)%len(flag_rma)],(screen.get_width()//2-100-flag_school_dim[0],screen.get_height()//2-flag_school_dim[1]//2))
    screen.blit(flag_x[(count)%len(flag_x)],(screen.get_width()//2+100,screen.get_height()//2-flag_school_dim[1]//2))

    versus = font_30.render("VS",True,WHITE)
    versus_rect = versus.get_rect()
    versus_rect.center = (screen.get_width()//2,screen.get_height()//2)
    screen.blit(versus,versus_rect)
    
    screen.blit(boat,(count,screen.get_height()//2-boat.get_height()//2-175))
    screen.blit(flag_be[(count)%len(flag_be)],(count-10,screen.get_height()//2-162))
    
    screen.blit(boat,(count,screen.get_height()//2-boat.get_height()//2+75))
    screen.blit(flag_fr[(count)%len(flag_fr)],(count-10,screen.get_height()//2+90))
    
    score1 = font_50.render("DISTANCE: "+str(count) + " m", True, color)
    score1_rect = score1.get_rect()
    score1_rect.midtop = (screen.get_width()//2,100)

    score2 = font_50.render("DISTANCE: "+str(count) + " m", True, WHITE)
    score2_rect = score2.get_rect()
    score2_rect.midbottom = (screen.get_width()//2,screen.get_height()-100)

    time_left = 10*60-round(duration)
    timer = font_70.render((time_left<60*10)*"0"+str(time_left//60)+":"+(time_left%60<10)*"0"+str(time_left%60),True,WHITE)
    timer_rect = timer.get_rect()
    timer_rect.center = (screen.get_width()//2,100)
    screen.blit(timer,timer_rect)

    pg.display.flip()
    clock.tick(30)
    screen.fill(BLACK)
    count += 1
pg.quit()


In [ ]:
from pyrow import pyrow as pr
from threading import Thread

In [ ]:
devices = [pr.PyErg(erg) for erg in list(pr.find())]
threads = []
for device in devices:
    threads.append(Thread(target=device.get_monitor))

In [ ]:
def monitor_stream(device:pr.PyErg):
    while True:
        device.get_monitor()